In [40]:
import tweepy
import nltk
import pandas as pd
import pickle
import numpy as np 
import re 
import os
import random
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
from statistics import mode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [8]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [9]:
# Open dataset 
dataset = pd.read_csv('../resource/labeled_data.csv')
# class: 0 - hate speech 1 - offensive language 2 - neither
dt_transformed = dataset[['class', 'tweet']]

In [10]:
# df_train, df_test = train_test_split(dt_transformed, test_size = 0.15, random_state = 42, stratify=dt_transformed['class'])
# print(f"""
# Training table have {df_train.shape[0]} rows and {df_train.shape[1]} column 
# Test table have {df_test.shape[0]} rows and {df_test.shape[1]} column
# """)
# df_train['class'].value_counts().plot(kind='barh',color='c');df_test['class'].value_counts().plot(kind='barh',color='m')
dt_transformed['class'].values[0]

2

In [11]:
all_words = [] 
documents = [] 

def processing_text_to_pos_tag(row_table):
    all_stopwords = stopwords.words('english')
    t_data = re.sub("^RT.", "", row_table[1])  # removing RTT
    t_data = re.sub("@[0-9A-za-z:]+", "", t_data)  # removing @user
    t_data = re.sub("https?://[A-Za-z0-9./&#]+", "", t_data)  # removing http links
    t_data = re.sub("&#[0-9;]+", " ", t_data) # removing emojis
    t_data = t_data.lower()
    t_data = t_data.split()
    ps = PorterStemmer()
    t_data = [ps.stem(word) for word in t_data if not word in set(all_stopwords) if len(word) > 2]
    t_data = ' '.join(t_data)
    row_table[0] = int(row_table[0])
    if row_table[0] == 0:
        row_table[0] = 1
    return row_table[0], t_data

In [12]:
# allowed_word_types = ["J"]

for row in dt_transformed.values:
    c,t = processing_text_to_pos_tag(row)
    documents.append( (t,c) )
    words = word_tokenize(t)
#     pos_t = nltk.pos_tag(words)
    for w in words:
        all_words.append(w)
        
save_documents = open("../pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:5000]

save_word_features = open("../pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [13]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [14]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [15]:
# len(featuresets)
testing_set = featuresets[11000:]
training_set = featuresets[:11000]

In [16]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
# classifier.show_most_informative_features(15)

save_classifier = open("../pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

save_classifier = open("../pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

Original Naive Bayes Algo accuracy percent: 93.0494086918668
MNB_classifier accuracy percent: 92.96234491765219
BernoulliNB_classifier accuracy percent: 91.82326053834434
LogisticRegression_classifier accuracy percent: 95.5017049989117
LinearSVC_classifier accuracy percent: 94.48596096640789
SGDClassifier accuracy percent: 95.32757745048248


In [17]:
voted_classifier = VoteClassifier(
                                  classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

In [36]:
text_ = ' '
def out(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)

# c,cc = out(text_)
# if cc <= 0.8:
#     print('Sem certeza')
# else:
#     if c == 1:
#         print('Discurso de odio ou ofensiva')
#     else:
#         print('Normal')

In [62]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json


def out(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)


ckey="yQuXSscTQvVm2wZ8g3e8tYJrE"
csecret="GVky1dWPwyFz0eOo4JGx4TYG5aB7XkgGcoLBnntAV1KkoZkX9z"
atoken="2555813954-k0L7QV7ds9yuxm5vdUnr0yFhhZkEXX3dZtUKnpc"
asecret="vB3BZplnbCq9UdRbq1NUUJqtsOr2EnzVoiQ6scLw7WNJJ"


class listener(StreamListener):

    def on_data(self, data):

        all_data = json.loads(data)

        tweet = all_data["text"]
        sentiment_value, confidence = out(tweet)
        print(tweet, sentiment_value, confidence)

        if confidence*100 >= 80:
            output = open("twitter-out.txt","a")
            output.write(str(sentiment_value))
            output.write('\n')
            output.close()

        return True

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["happy"])

RT @EliMcCann: Going to start randomly telling neighbors “I don’t care about all the stupid gossip. I’m happy you live here.” 1 0.6
RT @0_0oksak0_0: happy jihyo day!!🌹♡ the most beautiful, talented and sweetest girl in the world, jihyo, I love you♥️✨
#jihyoday #twice htt… 2 1.0
@craigcartonlive Happy bday Craigy! My man pots and pans 1 1.0
I love you Harry and I hope you will always do what makes you happy with the people who make you happy, thank you a… https://t.co/ybbMVJfpnM 2 1.0
RT @redsunO21: :) = im happy 
:] = hehehe.... hohoho... 2 1.0
RT @jihyoIoops: happy birthday jihyo ❤️ thank you for all your hardwork and positivity, the endless love, happiness and light that you brin… 2 0.6
RT @JYPETWICE: Happy Birthday JIHYO

#HappyJIHYOday https://t.co/aGKwBMVzZX 2 0.6
@Terrri__ Happy birthday 💕 1 0.6
RT @chloexhalle: happy sunday 🤍 https://t.co/D1LEkVkhgy 2 0.8
RT @chloexhalle: happy sunday 🤍 https://t.co/D1LEkVkhgy 2 0.8
@RasmalaizMegh I am so relieved! I really had low expectations 

RT @chloexhalle: happy sunday 🤍 https://t.co/D1LEkVkhgy 2 0.8
@bensemchee Happy birthday Lacy loo! 🎉🎈🐶 1 0.6
@nasatzupink @scorpyeon @JYPETWICE Thankyou for the give away, Happy Birthday to our leader💛 (jihyo is my 2nd bias… https://t.co/IYWeaAwhZE 2 0.8
RT @ItsKAYLAhoe__: Tyler Perry said before you marry someone you need to see them in all 5 seasons 🗣 When they HAPPY, MAD, STABLE, BROKE, &amp;… 1 0.6
@carlypug Happy Sunday Carly ♥️ 1 0.6
RT @NobleCell_Image: Happy Sunday folks. Just thought I'd cheer you up by sharing some of our favorite microscopy images #sciart https://t.… 2 0.8
@madisonbeer happy one year of good im goodbye 🥺 1 0.6
RT @Dream: Happy pride month! Proud to support anyone from the LGBTQ+ community :) I love you all ❤️ 2 1.0
RT @tellietubbs: Jjk 137

I ain’t reading all that. Happy for u tho Gege. Or sorry that happened. https://t.co/tzmaqzgMeE 2 0.8
@bastianlveiza happy birthday loser 🎂 misss u. hope ur doing good (‘: 2 1.0
RT @BernadetteClaxt: @SanjaHirkic @sol_mari

RT @babyjazzy09: happy titty day😋 https://t.co/oF8MWm77zU 2 0.6
RT @JYPETWICE: Happy Birthday JIHYO

#HappyJIHYOday https://t.co/aGKwBMVzZX 2 0.6
RT @PopHorrorNews: Happy 50th birthday to Patricia Velasquez, star of THE MUMMY, THE MUMMY RETURNS, THE CURSE OF LA LLORONA, and more! http… 2 1.0
RT @Auntyadaa: This year has started so well for @djkaywise and I am so happy for him. He just bagged an endorsement deal with @goldberglag… 2 1.0
not that i had a choice but i'm looking at a boy in the mirror i'm so happy :)) 2 1.0
RT @hlnlzdaily: 🎉| Here's our last suprise video for you guys! Happy Walls Day and we hope you enjoy the video! #1YearOfWalls https://t.co/… 2 1.0
RT @_MAJ004: #HBD_Precious_Ayesha

You’re quite different from me but you’re the closest to my heart. Thank you for being the only person w… 2 1.0
Happy Birthday @Divincenzo https://t.co/AHhPenylY3 1 0.6
@Gr3Te4rights @sweetsuthnpeach @DavidLevy101 @PopGpp @aime2smile @SuzanneHuse @4_AlwaysLoyal @BabsR10 @MillarBreen… https:/

@biolakazeem Was so happy when I heard he was going on loan then. 2 1.0
RT @CallmeA18: Happy Halloween Guys 🥳🥳🥳, this boy will be fucked by his roommate a poltergeist ghost
#NSFW #bara #yaoi #gay https://t.co/zJ… 1 0.8
@EGNzi51EcKbDmDl 植木野さん

お早うございます(⌒0⌒)／~~

今日から２月に入りました。
年末年始からずっとお仕事をされていますが、植木野さんはちゃんとお休みしているのですか？
今日はお休みです。と聞いた記憶… https://t.co/lu3PiWuRXF 2 1.0
appreciate it bro ✊🏾 1 1.0
RT @lisashots: she's so excited and happy i'm about to cry https://t.co/GEmsdBXafc 2 1.0
@BadCoEnt happy birthday jdawg :)) 2 1.0
me for the first 3ish yrs of deukae existing 😩 1 0.6
@1dBusybee HAPPY BIRTHDAY HARRY
WE LOVE YOU HARRY
#WELOVEYOUHARRYFROMIRAN 1 0.6
happy birthday golden boy, you are so special 🥰🥰🥰🥰 #happybirthdayharrystyles @Harry_Styles https://t.co/qAWhBjaHRZ 2 1.0
RT @_Andrae: Lowkey scared to get real happy cause when I get too happy shit always go wrong 1 1.0
RT @JYPETWICE: Happy Birthday JIHYO

#HappyJIHYOday https://t.co/aGKwBMVzZX 2 0.6
RT @_paakwesi_: I'm a virgin 
Sakawa boys 

Hubs got me a new Rick and Morty rig I'm so in love. 😍💖 Its smaller than my sunshine bong and this percolator is so… https://t.co/0xasWHv0LB 2 0.8
@spaceprincesses that makes me happy /g 1 0.6
RT @shoe0nhead: @KyleKulinski happy birthday 🎊😊 2 0.6
RT @ghana_recipe: ❤️HAPPY BIRTHDAY 27th❤️

いつもトキメキを有難うございます！
実はガーナと同じお誕生日の吉沢さん🥺
日頃の感謝を込めて、 #ガーナハート の #吉沢亮 さんを描いてみました…♡

#吉沢亮誕生祭 #ガーナ #ガーナの日… 2 0.6
RT @waitbutwhy: “If you only wished to be happy, this could be easily accomplished; but we wish to be happier than other people, and this i… 2 1.0
Happy Sunday. #dragonage #da2 #fenris #fanart https://t.co/9kzba4aQcl 2 0.8
RT @fiuchicks: Why isn’t it a surprise that this is FAU...

Grow up and move on sis. Jealousy is an ugly trait and the sole fact that your… 2 1.0
RT @JYPETWICE: Happy Birthday JIHYO

#HappyJIHYOday https://t.co/aGKwBMVzZX 2 0.6
RT @salaleyoo: happy win metawin’s month 🐰👶🏻🎉🤏🏻💚
#winmetawin

(📸wmo) https://t.co/FDw7hYnrIx 2 1.0
RT @moonbtea: HAPPY SOLAR MONTH #OurSunshineSolar

#SOLA

RT @VolkorNSFW: 🖤 Dva Fingers Tracer 🖤
🎞️ Animation by @Sage_Of_Osiris
🎧 Audio by @VolkorNSFW 🧙🏼‍♂️✨
🎤 Voices by @PeachyMizz &amp; @_PixieWillo… 1 0.6
RT @chloexhalle: happy sunday 🤍 https://t.co/D1LEkVkhgy 2 0.8
By the by, if you want to donate and get some cookies in return (or give them to troops and the Regional Food Bank)… https://t.co/HhrpcScd1L 2 0.6
@NAONAO_Kloves NAONAO ﾁｬﾝ おはよん　ｴｶﾞｵ
今日もﾜｸﾜｸケイを待つ　ﾊｱﾄ

k our hero
k ot5
K reliable
K rainbow 
K happy
K hamster owne… https://t.co/KCaVWffVt3 2 1.0
RT @hassanabkr_: Take time and do what makes your soul happy https://t.co/KxNMtJx8EY 2 0.8
RT @sogandx: Happy birthday!!
WE LOVE YOU HARRY
#WELOVEYOUHARRYFROMIRAN https://t.co/9whljKMQdK 2 0.6
Please help support HAPPY BIRTHDAY HARRY, add a #Twibbon now! https://t.co/1uQLxAN4lM 2 1.0
RT @tomverica: Wishing this truly remarkable human a very Happy Birthday. Much love and joy KW! ⁦@kerrywashington⁩ https://t.co/gEm1orTH5k 2 0.6
RT @dakotadaviis: Happy Sunday 🙂 https://t.co/364BwWQmIb 1 0.6


j’ai jamais été aussi heureux de mv 😜 1 1.0
RT @ezralevant: This looks like some fading memory from the Before Times. People happy and free, standing near each other, living as a comm… 2 1.0
@nasatzupink @scorpyeon @JYPETWICE Thankyou for the give away, Happy Birthday to our leader💛 (jihyo is my 2nd bias… https://t.co/8eVSbMDQgU 2 0.8
RT @lisashots: she's so excited and happy i'm about to cry https://t.co/GEmsdBXafc 2 1.0
RT @de4thangeI: yall better wish happy birthday to my gf jihyo🖤 happy birthday queen https://t.co/Z4BgT7nw8C 1 0.6
HAPPY BIRTHDAAAYYYYY @Harry_Styles https://t.co/7GTS15eZhJ 1 0.6
RT @cutejoker_0707: I know questions boht zyada derogatory the !! Bt the way our queen answered them was bang on !!
More power to u Rubi &amp;… 1 0.8
RT @eteroutsider: happy #secretsamol to @ratvortex !!
i was thinking about Milli and Stray Dog myself so I had a lot of fun figuring out so… 2 1.0
RT @JYPETWICE: Happy Birthday JIHYO

#HappyJIHYOday https://t.co/aGKwBMVzZX 2 0.6
If u think faki

KeyboardInterrupt: 